<h1><span style="color:red">Add Color Statistics for Images in Image Collection</span></h1>

### This sample notebook will read survey images and add lightness, hue, brightness, saturation, and RGB values to a new survey version

## 1. Retrieve survey parameters from the URL

In [58]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Read the survey file and navigate to full-size images

In [59]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from PIL import Image, ImageStat
import glob, os
import csv
import pandas as pd
from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

# remember the current directory
# curdir = os.getcwd()
absolutePath = "../../temp_csvs/"

# read the csv file
file = open(absolutePath + csv_file, encoding="latin-1")
df = pd.read_csv(file)
dflen = len(df.columns)

localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")


In [60]:
# include RGB values
RGB = True
# include Lightness values
Light = True
# include rms values
rms = False
printmd("<b><span style='color:red'>The following color characteristics will be added:</span></b>")
if RGB:
    print('Red-Green-Blue bands: Mean, Median, Standard Deviation: YES')
if Light:
    print('Lightness: Mean, Median, Standard Deviation: YES')
if rms:
    print('Root-Mean-Square for the above bands: : YES')

printmd("<b><span style='color:red'>Modify and rerun this cell to adjust</span></b>")


<b><span style='color:red'>The following color characteristics will be added:</span></b>

Red-Green-Blue bands: Mean, Median, Standard Deviation: YES
Lightness: Mean, Median, Standard Deviation: YES


<b><span style='color:red'>Modify and rerun this cell to adjust</span></b>

## 3. Specify band statistics functions ##

In [61]:
def lightness( im_file ):
    im = Image.open(im_file).convert('L')
    stat = ImageStat.Stat(im)
    return [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]]

def RGBstats ( im_file ):
    im = Image.open(im_file).convert('RGB')
    stat = ImageStat.Stat(im)
    return [
        [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]],
        [stat.mean[1], stat.median[1], stat.rms[1], stat.stddev[1]],
        [stat.mean[2], stat.median[2], stat.rms[2], stat.stddev[2]]
    ]
def HSVstats ( im_file ):
    im = Image.open(im_file).convert('HSV')
    stat = ImageStat.Stat(im)
    return [
        [stat.mean[0], stat.median[0], stat.rms[0], stat.stddev[0]],
        [stat.mean[1], stat.median[1], stat.rms[1], stat.stddev[1]],
        [stat.mean[2], stat.median[2], stat.rms[2], stat.stddev[2]]
    ]

## 4. Process images (will take a while)

In [62]:
# Processing image file and adding to data frame
all_data = []
counter = 0
a = widgets.Label(value="0% done")
display(a)

# Uncomment only if want to process based on level 8 tiles. It is faster but not as accurate.
# for dzi in glob.glob("*.dzi"):
#     file = os.path.join(os.getcwd(), dzi[:-4] +"_files", "8", "0_0.jpg")

# numfiles = len(glob.glob("*.png"))
numfiles = len(glob.glob(full_images_location+"*.png"))

for file in glob.glob(full_images_location+"*.png"):
    file_data = {} # data for this particular file
    try:
        file_data['#img'] = os.path.basename(file)[:-4]
        file_data['Lightness_mean']   = lightness(file)[0]
        file_data['Lightness_median'] = lightness(file)[1]
        file_data['Lightness_rms']    = lightness(file)[2]
        file_data['Lightness_std']    = lightness(file)[3]
        file_data['Hue_mean']          = HSVstats(file)[0][0]
        file_data['Hue_median']        = HSVstats(file)[0][1]
        file_data['Hue_rms']           = HSVstats(file)[0][2]
        file_data['Hue_std']           = HSVstats(file)[0][3]
        file_data['Saturation_mean']   = HSVstats(file)[1][0]
        file_data['Saturation_median'] = HSVstats(file)[1][1]
        file_data['Saturation_rms']    = HSVstats(file)[1][2]
        file_data['Saturation_std']    = HSVstats(file)[1][3]
        file_data['Brightness_mean']        = HSVstats(file)[2][0]
        file_data['Brightness_median']      = HSVstats(file)[2][1]
        file_data['Brightness_rms']         = HSVstats(file)[2][2]
        file_data['Brightness_std']         = HSVstats(file)[2][3]
        file_data['Red_mean']          = RGBstats(file)[0][0]
        file_data['Red_median']        = RGBstats(file)[0][1]
        file_data['Red_rms']           = RGBstats(file)[0][2]
        file_data['Red_std']           = RGBstats(file)[0][3]
        file_data['Green_mean']        = RGBstats(file)[1][0]
        file_data['Green_median']      = RGBstats(file)[1][1]
        file_data['Green_rms']         = RGBstats(file)[1][2]
        file_data['Green_std']         = RGBstats(file)[1][3]
        file_data['Blue_mean']         = RGBstats(file)[2][0]
        file_data['Blue_median']       = RGBstats(file)[2][1]
        file_data['Blue_rms']          = RGBstats(file)[2][2]
        file_data['Blue_std']          = RGBstats(file)[2][3]
        all_data.append(file_data)    
    except Exception as e:
        print(file, "There was an issue: ", e)
    counter += 1
    a.value = str(int(counter / numfiles * 100)) + "% done"
#debugging
    #display(a)
        #print(str(counter)," ::  Processed file: ", file)
    #if counter == 30:
    #      break
        
newdf = pd.DataFrame(all_data).fillna('')
printmd("<b><span style='color:red'>All files processed</span></b>")

Label(value='0% done')

<b><span style='color:red'>All files processed</span></b>

## 5. Add the result to the survey dataframe ##

In [63]:
columnTitles = [
    '#img',
    'Brightness_mean','Brightness_median']
if rms:
    columnTitles.append('Brightness_rms')
columnTitles.extend(('Brightness_std','Hue_mean','Hue_median'))
if rms:
    columnTitles.append('Hue_rms')
columnTitles.extend(('Hue_std','Saturation_mean','Saturation_median'))
if rms:
    columnTitles.append('Saturation_rms')
columnTitles.append('Saturation_std')

if Light:
    columnTitles.extend(('Lightness_mean', 'Lightness_median'))
    if rms: 
        columnTitles.append('Lightness_rms')
    columnTitles.append('Lightness_std')
    
if RGB:
    columnTitles.extend(('Red_mean','Red_median'))
    if rms:
        columnTitles.append('Red_rms')
    columnTitles.extend(('Red_std','Green_mean','Green_median'))
    if rms:
        columnTitles.append('Green_rms')
    columnTitles.extend(('Green_std','Blue_mean','Blue_median'))
    if rms:
        columnTitles.append('Blue_rms')
    columnTitles.append('Blue_std')

printmd("<br><b><span style='color:red'>The following columns will be added:</span></b>")    
print(columnTitles[1:])

newdf = newdf[columnTitles]


<br><b><span style='color:red'>The following columns will be added:</span></b>

['Brightness_mean', 'Brightness_median', 'Brightness_std', 'Hue_mean', 'Hue_median', 'Hue_std', 'Saturation_mean', 'Saturation_median', 'Saturation_std', 'Lightness_mean', 'Lightness_median', 'Lightness_std', 'Red_mean', 'Red_median', 'Red_std', 'Green_mean', 'Green_median', 'Green_std', 'Blue_mean', 'Blue_median', 'Blue_std']


In [64]:
# add #number to numeric colomn names

collist = []
for col in newdf.columns:
    col1 = col+"#number"
    collist.append(col1)
newdf.columns = collist
newdf = newdf.rename(columns={'#img#number': '#img'})
df = pd.merge(df, newdf, on='#img', how='outer')
printmd("<b><span style='color:red'>Dataframe created</span></b>")

<b><span style='color:red'>Dataframe created</span></b>

## 6. Save the new version of CSV file, and give a name to new survey

In [65]:
new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)

<b><span style='color:red'>A new temporary file will be created at: </span></b>

../../temp_csvs/public_Picasso_Paintings_v1.csv


In [66]:
#Input survey name

from IPython.display import display
input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>

Text(value='')

Text(value='')

In [67]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

<b><span style='color:red'>Survey Name is: </span></b>picolo2

## 7. Generate the survey and create survey URL

In [68]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}
upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + input_text.value + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)



<b><span style='color:red'>New survey created successfully</span></b>

https://suave-dev.sdsc.edu/main/file=public_picolo2.csv&views=1110101&view=grid


<b><span style='color:red'>Click the URL to open the new survey</span></b>